In [ ]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.data.loader import *
from model_based_mvpa.models.model import *

In [ ]:
root = '/data2/project_model_based_fmri/ds001882/'

In [ ]:
layout = BIDSLayout(root, derivatives=True)

## PREPROCESS FMRI IMAGE

In [ ]:
X, voxel_mask, layout = bids_preprocess(layout=layout, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=2, nthread=3)

In [ ]:
#todo: maskdata visualization

## PREPRROCESS EVENT DATA

In [ ]:
def example_piva_adjust_columns(row):
    ## rename data in a row to the name which can match hbayesdm.dd_hyperbolic requirements ##
    if row["delay_left"] >= row["delay_right"]:
        row["delay_later"] = row["delay_left"]
        row["delay_sooner"] = row["delay_right"]
        row["amount_later"] = row["money_left"]
        row["amount_sooner"] = row["money_right"]
        row["choice"] = 1 if row["choice"] == 1 else 0
    else:
        row["delay_later"] = row["delay_right"]
        row["delay_sooner"] = row["delay_left"]
        row["amount_later"] = row["money_right"]
        row["amount_sooner"] = row["money_left"]
        row["choice"] = 1 if row["choice"] == 2 else 0
    return row

In [ ]:
def example_piva_condition(row):
    # in the paper, the condition for trial varies in a single run,
    # agent == 0 for making a choice for him or herself
    # agent == 1 for making a choice for other
    # to consider only non-social choice behavior, select only the cases with agent == 0
    return row["agent"] == 0

In [ ]:
def example_piva_modulation(row, param_dict):
    # calculate subjective utility for choosing later option over sooner option
    # hyperbolic discount function is adopted
    ev_later = row["amount_later"] / (1 + param_dict["k"] * row["delay_later"])
    ev_sooner  = row["amount_sooner"] / (1 + param_dict["k"] * row["delay_sooner"])
    modulation = ev_later - ev_sooner
    row["modulation"] = modulation
    return row

In [ ]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(layout=layout,
                      all_individual_params_path="/data2/project_model_based_fmri/ds001882/derivatives/fmriprep/mvpa/all_individual_params.tsv",
                      preprocess=example_piva_adjust_columns,
                      condition=example_piva_condition,
                      modulation=example_piva_modulation,
                      dm_model="dd_hyperbolic")

## LOAD DATA

In [ ]:
import nibabel as nib
from pathlib import Path

X, y = prepare_dataset(root=layout.derivatives["fMRIPrep"].root)
voxel_mask = nib.load(Path(layout.derivatives["fMRIPrep"].root) / 'mvpa' / 'voxel_mask.nii.gz')

In [ ]:
X.shape, y.shape

## FIT MVPA MODEL

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
n = 30
coefs = mlp_regression(X, y,
                       layout=layout,
                       layer_dims=[1024, 1024],
                       N=n,
                       verbose=1)

task_name = f'piva2019_mlp_{n}'
result = get_map(coefs, voxel_mask, task_name=task_name, map_type='z', save=True, save_path='./results', sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
alpha = 0.005
coefs = elasticnet(X, y,
                   alpha=alpha,
                   n_jobs=16,
                   verbose=1,
                   max_lambda=1,
                   n_samples=5000)

task_name = f'piva2019_elasticnet_{n}_{alpha:.4f}'
result = get_map(coefs, voxel_mask, task_name=task_name, map_type='z', save=True, save_path='./results', sigma=1)
print(time.strftime('%c', time.localtime(time.time())))